# This notebook is the first notebook of code for the project e-Pragya, this project is intented to build up from the scratch, starting from the basic encoder decoder model and going all the way to where the current Status quo of Ai stands at

In [ ]:
!pip install pytorch

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

folder_path = '/kaggle/input/personal-notes'
text_list = []

# Looping through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            text_list.append(content)

# the list text_data_list contains all the text data


In [ ]:
text = '\n'.join(text_list)

In [ ]:
print("Total number of characteres in the input text", len(text))

In [ ]:
print("lets look at the first 1000 characters", text[:1000])

In [ ]:
#so we have the text, now first take the unique characters from the text
#and then put them in the list and then sort them, calaculate the length
#list and then print all of the information

In [ ]:
char = sorted(list(set(text)))
number_of_char = len(char)
print(''.join(char))
print("number of characters is", number_of_char)

In [ ]:
stoi = { ch:i for i,ch in enumerate(char)}
itos = { i:ch for i,ch in enumerate(char)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [''.join(itos[i] for i in l)]

first i made an mapping between number and characters and then from character to numbers
then i created the function which returns the numbers for the given string, thus encoding
then i created the function which returns the characters for the given numbers, thus decoding

now lets convert the list of numbers into tensors, cause pytorch works in tensors

In [ ]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

now convert the data into training and validation set

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train[:block_size+1]

now lets define the context and target values, all the values are the context data while every value except for the first one becomes target, cause target is the thing our model computes after it looks at the context, its the thing that comes next,so first word isnt target cause there is no context before the first value

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context=x[:i+1]
    target=y[i]
    print(f"When the context is {context} target is {target}")

so now create a batch of training and testing data 
gpu enable us to stact multple data into a batch and process it as a whole

In [ ]:
torch.manual_seed(2058)
batch_size = 4
block_size = 8

#now lets take the random index from the train and test data and stack them
def get_batch(split):
    data = train_data if split=='train'else val_data
    ix= torch.randint(len(data)-block_size,batch_Size)